# 자막 추출
### pip install openai

In [13]:

import openai
import pandas as pd
import os


In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")  # 또는 직접 입력 가능

In [16]:
# 자막 DataFrame 불러오기
transcript_df = pd.read_csv("../data/transcript_data.csv")
transcript_df.dropna(subset=["transcript"], inplace=True)

In [ ]:
CATEGORY_CLASSIFY_PROMPT_KO = """
다음은 유튜브 영상의 자막입니다.
내용을 분석한 뒤, 아래 카테고리 중 가장 적절한 하나를 선택해서 반환하세요.

- 뉴스
- 엔터테인먼트
- 사건/사고
- 사회이슈
- 기타

※ 설명은 하지 말고, 카테고리명만 정확하게 출력하세요.

자막:
{text}
"""

In [18]:
def classify_category_ko(transcript):
    try:
        prompt = CATEGORY_CLASSIFY_PROMPT_KO.format(text=transcript[:1500])
        res = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.2,
            max_tokens=10
        )
        return res["choices"][0]["message"]["content"].strip()
    except Exception as e:
        print("오류:", e)
        return "기타"


In [19]:
STRUCTURED_SUMMARY_PROMPT_KO = """
다음은 유튜브 영상의 자막입니다.
이 자막을 기반으로 다음과 같은 구조로 깔끔하게 요약해 주세요:

1. 먼저, 영상 전체 내용을 한 문장으로 요약해 주세요 (인트로 요약).
2. 이후에는 각 핵심 주제를 번호(1, 2, 3...)로 나누어 정리해 주세요.
3. 각 항목은 [소제목] + 내용 설명 형식으로 작성해 주세요.
4. 불필요한 대화나 반복은 생략하고, 개념, 사례, 시사점을 중심으로 요약해 주세요.
5. 너무 길게 쓰지 말고, 한 항목당 2~4문장 정도로 압축해 주세요.

자막:
{transcript}
"""

In [20]:
def summarize_structured_with_category_ko(transcript):
    category = classify_category_ko(transcript)
    prompt = STRUCTURED_SUMMARY_PROMPT_KO.format(transcript=transcript[:3000])

    try:
        res = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5,
            max_tokens=700
        )
        return category, res["choices"][0]["message"]["content"].strip()
    except Exception as e:
        print("요약 실패:", e)
        return category, None

In [21]:
summaries = []

In [22]:
for idx, row in transcript_df.iterrows():
    category, summary = summarize_structured_with_category_ko(row["transcript"])
    summaries.append({
        "video_id": row["video_id"],
        "title": row["title"],
        "url": row["url"],
        "channel": row["channel"],
        "published": row["published"],
        "category": category,
        "summary": summary
    })

In [23]:
summary_df = pd.DataFrame(summaries)
summary_df.to_csv("../data/summaries.csv", index=False, encoding="utf-8-sig")
summary_df.head()

,video_id,title,url,channel,published,category,summary
0,iwUYHUPwL30,ADsP 핵심요약 5시간 벼락치기 특강 | 미어캣 [편집본] | adsp 독학,https://www.youtube.com/watch?v=iwUYHUPwL30,비빅터,1 year ago,기타,"인트로: 데이터의 특성은 존재적 특성과 단위적 특성으로 나뉘며, 존재적 특성은 팩트..."
1,m6Vez0J2Xig,ADsP 핵심요약 5시간 벼락치기 특강 _live stream 녹화,https://www.youtube.com/watch?v=m6Vez0J2Xig,위키북스,Streamed 2 years ago,엔터테인먼트,"인트로 요약: 오디오 문제로 죄송하다며 강의 시작, adsp 시험 대비 전략 소개\..."
2,8YN9vZp495E,[ADsP 1과목] 데이터의 이해 - 기다리던 총정리 시간,https://www.youtube.com/watch?v=8YN9vZp495E,EduAtoZ - Programming,3 years ago,기타,인트로: 데이터의 정의와 유형에 대해 쭉 정리하는 시간을 갖도록 하겠습니다.\n\n...
3,_ZrneQl69Lo,ADsP 핵심이론 5시간 완성 ｜ 시험 전 벼락치기 필수 영상🔥 ｜ 와우패스 김계철...,https://www.youtube.com/watch?v=_ZrneQl69Lo,와우패스,2 years ago,기타,인트로: 2023년 adsp 항공으로 끝내기 요약 플러스 문제집 파트에는 185개의...
4,MuJUhXoM-9E,[ADsP 3과목] 데이터 분석 - Part 3 - 기다리던 총정리 시간,https://www.youtube.com/watch?v=MuJUhXoM-9E,EduAtoZ - Programming,3 years ago,기타,인트로: 데이터 마이닝은 데이터를 기반으로 감춰진 정보를 발견하여 비즈니스 의사 결...


In [24]:
print(summary_df)

      video_id                                              title  \
0  iwUYHUPwL30        ADsP 핵심요약 5시간 벼락치기 특강 | 미어캣 [편집본] | adsp 독학   
1  m6Vez0J2Xig              ADsP 핵심요약 5시간 벼락치기 특강 _live stream 녹화   
2  8YN9vZp495E                   [ADsP 1과목] 데이터의 이해 - 기다리던 총정리 시간   
3  _ZrneQl69Lo  ADsP 핵심이론 5시간 완성 ｜ 시험 전 벼락치기 필수 영상🔥 ｜ 와우패스 김계철...   
4  MuJUhXoM-9E           [ADsP 3과목] 데이터 분석 - Part 3 - 기다리던 총정리 시간   

                                           url                channel  \
0  https://www.youtube.com/watch?v=iwUYHUPwL30                    비빅터   
1  https://www.youtube.com/watch?v=m6Vez0J2Xig                   위키북스   
2  https://www.youtube.com/watch?v=8YN9vZp495E  EduAtoZ - Programming   
3  https://www.youtube.com/watch?v=_ZrneQl69Lo                   와우패스   
4  https://www.youtube.com/watch?v=MuJUhXoM-9E  EduAtoZ - Programming   

              published category  \
0            1 year ago       기타   
1  Streamed 2 years ago   엔터테인먼트   
2           3 years ago       기타   
3 